# 한국어 문장 관계 분류 경진대회

#### 한 쌍의 문장은 Premise와 Hypothesis로 구성되어 있습니다.

#### premise 문장을 참고해 hypothesis 문장이 참인지(Entailment), 거짓인지(Contradiction), 
#### 혹은 참/거짓 여부를 알 수 없는 문장인지(Neutral)를 판별해야 합니다.

#### 대회에 참가해 한국어 문장 관계를 파악할 수 있는 알고리즘을 작성해 주세요.

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
import sklearn
import konlpy

import os
import re
import tqdm
import urllib.request
from collections import Counter

from konlpy.tag import Okt 

In [21]:
!pip install konlpy==0.5.2

### 데이터 불러오기

In [3]:
df_train = pd.read_csv('train_data.csv', encoding = 'utf-8')
df_train

# premise 문장을 참고해 hypothesis 문장이 참인지 거짓인지 중립인지 판단하자

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
5,5,이번 증설로 코오롱인더스트리는 기존 생산량 7만7000톤에서 1만6800톤이 늘어나...,코오롱 인더스트리는 총 9만 3800톤의 생산 능력을 확보했다.,entailment
6,6,자신뿐만 아니라 남을 돕고자 하는 청년의 꿈과 열정에 모두가 주목하고 있다.,모든 청년은 꿈과 열정을 가지고 있다.,neutral
7,7,시대상황을 고려하는 현명한 시청태도가 요구되다.,시청태도에 특별한 주의점은 없다.,contradiction
8,8,사진과 차이없는 아기자기한 실내소품들과 분위기가 멋졌습니다.,아기자기한 실내소품들은 사진에서 본 것과 차이가 있었습니다.,contradiction
9,9,"빠른 답장과 간편한 체크인, 깨끗한 집 좋았어요",체크인이 복잡했어요.,contradiction


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24998 entries, 0 to 24997
Data columns (total 4 columns):
index         24998 non-null int64
premise       24998 non-null object
hypothesis    24998 non-null object
label         24998 non-null object
dtypes: int64(1), object(3)
memory usage: 781.3+ KB


In [5]:
df_train['label'].value_counts()

# entailment : 참
# contradiction : 거짓
# neutral : 중립

entailment       8561
contradiction    8489
neutral          7948
Name: label, dtype: int64

In [8]:
df_train2 = df_train[['index','premise']]
df_train2

,index,premise
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이..."
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,..."
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는..."
5,5,이번 증설로 코오롱인더스트리는 기존 생산량 7만7000톤에서 1만6800톤이 늘어나...
6,6,자신뿐만 아니라 남을 돕고자 하는 청년의 꿈과 열정에 모두가 주목하고 있다.
7,7,시대상황을 고려하는 현명한 시청태도가 요구되다.
8,8,사진과 차이없는 아기자기한 실내소품들과 분위기가 멋졌습니다.
9,9,"빠른 답장과 간편한 체크인, 깨끗한 집 좋았어요"


In [ ]:
### premise와 hypothesis 분석 -> 유사도 분석 진행해보기

In [14]:
def tokenize_and_stemming(sentence, tokenizer):

    sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence) 
    
    raw_pos_tagged = tokenizer.pos(sentence, stem=True) # POS Tagging with stemming

    sentence_tokenized = []
    for token, pos in raw_pos_tagged: # ex) word <- ('서울', 'Noun')
        # pos 기준 제거 대상 : ["Josa", "Eomi", "Punctuation", "Foreign", "Number", "Suffix", "Determiner"]
        # 리뷰 텍스트 상 형태소의 순서를 고려하는 경우 의미를 갖게 됨 : "VerbPrefix" (잘/안/못), "Conjunction"(하지만/그러나/그리고), "KoreanParticle"(ㅋㅋㅋ/ㅎㅎㅎ/ㅠㅠ)
        if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
            sentence_tokenized.append(token)  

    return sentence_tokenized

In [5]:
okt = Okt()
tokenized_sentences = []

for sentence in tqdm.tqdm(df_train2['premise']):
    try:
        tokenized_sentences.append(tokenize_and_stemming(sentence, okt))
    except:
        print("Error occured at :", sentence)
        tokenized_sentences.append([])

df_train2['premise'] = tokenized_sentences

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [6]:
import platform 
print(platform.architecture())

('64bit', 'WindowsPE')
